In [1]:
import sys
sys.path.append('..')

In [2]:
import os
import csv

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import TensorDataset
import pandas as pd

from defences.util import score
from models.numeric import NumericModel
from models.torch_util import predict_numpy

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
PATH_OUTPUTS = os.path.join('..', 'csv')
if not os.path.exists(PATH_OUTPUTS):
    os.makedirs(PATH_OUTPUTS)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

device: cuda


## Banknote

In [6]:
NAME_MODEL_FILE = 'breastcancer_400.pt'
N_FEATURES = 30
N_CLASSES = 2
model = NumericModel(
    n_features=N_FEATURES, 
    n_hidden=N_FEATURES * 4,
    n_classes=N_CLASSES,
    use_prob=True
)
path_model = os.path.join('..', 'results', NAME_MODEL_FILE)
model.load_state_dict(torch.load(path_model))
model = model.to(device)

In [7]:

name_data = 'breastcancer'
name_model = 'basic120'
name_attacks = [
    'apgd_0.05',
    'apgd_0.2',
    'apgd_0.4',
    'apgd_1.0',
    'apgd1_0.2',
    'apgd1_0.4',
    'apgd1_1.0',
    'apgd1_2.0',
    'apgd1_3.0',
    'apgd2_0.2',
    'apgd2_0.4',
    'apgd2_1.0',
    'apgd2_2.0',
    'apgd2_3.0',
    'boundary_0.3',
    'cw2_0.0',
    'cw2_10.0',
    'cw2_5.0',
    'cwinf_0.0',
    'cwinf_5.0',
    'deepfool_1e-06',
    'fgsm_0.05',
    'fgsm_0.2',
    'fgsm_0.4',
    'fgsm_1.0',
]
name_defences = [
'baard_2stage',
'baard_3stage',
'fs',
'lid',
'rc',
]

In [8]:
def get_dataframe(df, model, data, nmodel, nattack, ndefence):
    path_result = os.path.join('..', 'results', '{}_{}_{}_{}.pt'.format(data, nmodel, nattack, ndefence))
    result = torch.load(path_result)
    X_test = result['X_test']
    y_test = result['y_test']
    labels_test = result['labels_test']
    res_test = result['res_test']

    attack_eps = attack.split('_')

    idx_tp = np.where(labels_test == 1)[0]
    pred = predict_numpy(model, X_test[idx_tp], device=device)
    attack_score = np.sum(pred == y_test[idx_tp]) / len(idx_tp)
    my_score = score(res_test[idx_tp], y_test[idx_tp], pred, labels_test[idx_tp])

    block_rate = np.sum(res_test[idx_tp] == labels_test[idx_tp]) / len(idx_tp)

    idx_fp = np.where(labels_test == 0)[0]
    fpr = np.sum(res_test[idx_fp]) / len(idx_fp)
    if ndefence == 'rc':
        block_rate = -1
        fpr = np.sum(res_test[idx_fp] != y_test[idx_fp]) / len(idx_fp)

    df = df.append({
        'Attack': attack_eps[0], 
        'Epsilon': attack_eps[1], 
        'Defence': ndefence,
        'Accuracy after attack': attack_score * 100,
        'Score': my_score * 100,
        'Block Rate': block_rate * 100, 
        'False Positive Rate': fpr * 100
    }, ignore_index=True)
    return df

In [9]:
COLUMNS = ['Attack', 'Epsilon', 'Defence', 'Accuracy after attack', 'Score', 'Block Rate', 'False Positive Rate']

In [10]:
df = pd.DataFrame(columns=COLUMNS)

for attack in name_attacks:
    for defence in name_defences:
        try:
            df = get_dataframe(df, model, name_data, name_model, attack, defence)
        except FileNotFoundError:
            continue    

In [11]:
df.to_csv(os.path.join(PATH_OUTPUTS, '{}.csv'.format(name_data)))